In [2]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [6]:
df = sqlContext.read.json("/home/logic/Downloads/counter_sign.json")

In [5]:
df.select("scans").printSchema()

root
 |-- scans: struct (nullable = true)
 |    |-- ALYac: struct (nullable = true)
 |    |    |-- detected: boolean (nullable = true)
 |    |    |-- result: string (nullable = true)
 |    |    |-- update: string (nullable = true)
 |    |    |-- version: string (nullable = true)
 |    |-- AVG: struct (nullable = true)
 |    |    |-- detected: boolean (nullable = true)
 |    |    |-- result: string (nullable = true)
 |    |    |-- update: string (nullable = true)
 |    |    |-- version: string (nullable = true)
 |    |-- Acronis: struct (nullable = true)
 |    |    |-- detected: boolean (nullable = true)
 |    |    |-- result: string (nullable = true)
 |    |    |-- update: string (nullable = true)
 |    |    |-- version: string (nullable = true)
 |    |-- Ad-Aware: struct (nullable = true)
 |    |    |-- detected: boolean (nullable = true)
 |    |    |-- result: string (nullable = true)
 |    |    |-- update: string (nullable = true)
 |    |    |-- version: string (nullable = true)
 | 

In [ ]:
        T.MapType(
            T.StructField("detected",T.BooleanType(),True),
            T.StructField("result",T.StringType(),True), 
            T.StructField("update",T.StringType,True), 
            T.StructField("version",T.StringType,True))


In [ ]:
#adware
#pup

In [86]:
new_df = df.where("positives > 2").select(
    F.col("md5"), F.col("scans.*")
)
new_df.columns[1:]

new_df.withColumn(
    "detected_count", 
    sum([
        F.when(F.col(cl).getItem("detected"), 1).otherwise(0) for cl in new_df.columns[1:]
    ])
).select("md5", "detected_count").show()

+--------------------+--------------+
|                 md5|detected_count|
+--------------------+--------------+
|7ef3c7993f5d30075...|            41|
+--------------------+--------------+



In [133]:
new_df.withColumn(
    "file_type", 
    F.array([
        F.col(cl).getItem("result") for cl in new_df.columns[1:]
    ])
).withColumn(
    "file_type_count", 
    sum([
        F.when(
            F.instr(F.lower(F.col(cl).getItem("result")), "adware") > 0,
            1
        ).when(
            F.instr(F.lower(F.col(cl).getItem("result")), "pup") > 0,
            1
        ).otherwise(0) for cl in new_df.columns[1:]
    ])
).select(
    "md5", 
    F.explode("file_type").alias("file_type"),
    F.col("file_type_count")
).show(100,truncate=False)
""".where("file_type != 'null'").select(
    "md5", 
    F.when(
        F.lower(F.col("file_type"))
    )
).show(100,truncate=False)"""

+--------------------------------+-------------------------------------+---------------+
|md5                             |file_type                            |file_type_count|
+--------------------------------+-------------------------------------+---------------+
|7ef3c7993f5d30075432172cdd0c21da|Gen:Variant.Ursu.365454              |11             |
|7ef3c7993f5d30075432172cdd0c21da|Win32:Adware-gen [Adw]               |11             |
|7ef3c7993f5d30075432172cdd0c21da|suspicious                           |11             |
|7ef3c7993f5d30075432172cdd0c21da|Gen:Variant.Ursu.365454              |11             |
|7ef3c7993f5d30075432172cdd0c21da|null                                 |11             |
|7ef3c7993f5d30075432172cdd0c21da|Adware/Win32.Adposhel.R226766        |11             |
|7ef3c7993f5d30075432172cdd0c21da|null                                 |11             |
|7ef3c7993f5d30075432172cdd0c21da|null                                 |11             |
|7ef3c7993f5d30075432

'.where("file_type != \'null\'").select(\n    "md5", \n    F.when(\n        F.lower(F.col("file_type"))\n    )\n).show(100,truncate=False)'

In [134]:
new_df.withColumn(
    "file_type_count", 
    sum([
        F.when(
            F.instr(F.lower(F.col(cl).getItem("result")), "adware") > 0,
            1
        ).when(
            F.instr(F.lower(F.col(cl).getItem("result")), "pup") > 0,
            1
        ).otherwise(0) for cl in new_df.columns[1:]
    ])
).select(
    "md5", 
    F.col("file_type_count")
).show(100,truncate=False)

+--------------------------------+---------------+
|md5                             |file_type_count|
+--------------------------------+---------------+
|7ef3c7993f5d30075432172cdd0c21da|11             |
+--------------------------------+---------------+



In [ ]:
new_df.withColumn(
    "file_type", 
    F.array([
        F.col(cl).getItem("result") for cl in new_df.columns[1:]
    ])
).withColumn(
    "file_type_count", 
    sum([
        F.when(
            F.instr(F.lower(F.col(cl).getItem("result")), "adware") > 0,
            1
        ).when(
            F.instr(F.lower(F.col(cl).getItem("result")), "pup") > 0,
            1
        ).otherwise(0) for cl in new_df.columns[1:]
    ])
).select(
    "md5", 
    F.explode("file_type").alias("file_type"),
    F.col("file_type_count")
).show(100,truncate=False)


In [ ]:
new_df = df.where("positives > 2").select(
    F.col("md5"), F.col("positives"), F.col("scans.*")
)

new_df.withColumn(
    "file_type_count", 
    sum([
        F.when(
            F.instr(F.lower(F.col(cl).getItem("result")), "adware") > 0,
            1
        ).when(
            F.instr(F.lower(F.col(cl).getItem("result")), "pup") > 0,
            1
        ).otherwise(0) for cl in new_df.columns[2:]
    ])
).select(
    "md5", "positives",
    F.col("file_type_count")
).withColumn(
    "type",
    F.when(
        F.col("file_type_count") > (F.col("positives")/10),
        "pup"
    ).otherwise("virus")
).show(truncate=False)

Py4JJavaError: An error occurred while calling o23622.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 83.0 failed 1 times, most recent failure: Lost task 0.0 in stage 83.0 (TID 85, localhost): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.util.Arrays.copyOfRange(Arrays.java:2694)
	at java.lang.String.<init>(String.java:203)
	at java.lang.StringBuilder.toString(StringBuilder.java:405)
	at scala.StringContext.standardInterpolator(StringContext.scala:125)
	at scala.StringContext.s(StringContext.scala:90)
	at org.apache.spark.sql.catalyst.expressions.CaseWhen$$anonfun$toString$1.apply(conditionalExpressions.scala:216)
	at org.apache.spark.sql.catalyst.expressions.CaseWhen$$anonfun$toString$1.apply(conditionalExpressions.scala:215)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:328)
	at scala.collection.Iterator$class.foreach(Iterator.scala:727)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1157)
	

ERROR:py4j.java_gateway:Error while sending or receiving.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 746, in send_command
    raise Py4JError("Answer from Java side is empty")
Py4JError: Answer from Java side is empty
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.p

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.p

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.p

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.p

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.p

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.p

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.p

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.p

In [4]:
new_df = df.where("positives > 2").select(
    F.col("md5"), F.col("positives"), F.col("scans.*")
)

new_df.withColumn(
    "file_type_count", 
    sum([
        F.when(
            F.instr(F.lower(F.col(cl).getItem("result")), "adware") > 0,
            1
        ).when(
            F.instr(F.lower(F.col(cl).getItem("result")), "pup") > 0,
            1
        ).otherwise(0) for cl in new_df.columns[2:]
    ])
).select(
    "md5", "positives",
    F.col("file_type_count")
).withColumn(
    "type",
    F.when(
        F.col("file_type_count") > (F.col("positives")/10),
        "pup"
    ).otherwise("virus")
).show(truncate=False)

+--------------------------------+---------+---------------+----+
|md5                             |positives|file_type_count|type|
+--------------------------------+---------+---------------+----+
|7ef3c7993f5d30075432172cdd0c21da|41       |11             |pup |
+--------------------------------+---------+---------------+----+



In [11]:
new_df = df.select("md5", "additional_info").filter(
F.col("additional_info").getItem("sigcheck").getItem("verified") == "Signed"
).select(
	F.col("md5"),
	F.explode(
		F.col("additional_info").getItem("sigcheck").getItem("counter signers details")
	).alias("counter signers details")
).select(
	F.col("md5"),
	F.col("counter signers details").getItem("cert issuer").alias("cert issuer"),
    F.from_unixtime(F.unix_timestamp(F.substring(F.trim(F.col("counter signers details").getItem("valid from")),10,10), "MM/dd/yyyy"),"yyyy-MM-dd").alias("valid_from"),
    F.from_unixtime(F.unix_timestamp(F.substring(F.trim(F.col("counter signers details").getItem("valid to")),10,10), "MM/dd/yyyy"),"yyyy-MM-dd").alias("valid_to"),
).withColumn("difference",F.datediff(("valid_to"),("valid_from")))

AnalysisException: u'No such struct field verified in authentihash, comments, copyright, description, file version, link date, product;'

In [13]:
df.select("md5", F.col("additional_info").getItem("sigcheck"))

DataFrame[md5: string, additional_info[sigcheck]: struct<authentihash:string,comments:string,copyright:string,description:string,file version:string,link date:string,product:string>]

In [10]:
df.filter(
    F.col("additional_info").getItem("sigcheck").getItem("verified") == "Signed"
)

AnalysisException: u'No such struct field verified in authentihash, comments, copyright, description, file version, link date, product;'

In [ ]:
pd_frame.set_key("year").join(pd_frame_codesign.set_key("year"), lsuffix = "_left", rsuffix= "_right", how="outer")